## Hangman
In Hangman, you need to guess a word where all of its letters are hidden behind masks. In each turn, you name a letter and, if it appears in the word, all instances of it are unmasked. If it does not, you incur a penalty loss. You lose the game if your penalty reaches 6 before you guess the word and win otherwise.

### Strategy
The key to playing Hangman, to a large extent, lies in extracting patterns of natural language such as word structure and grammar. Thus, we adopt the NLP framework and attempt to solve the task with a BERT-like transformer. The vocabulary will consist of 26 English letters plus two special tokens: mask and padding; the output layer is a classification head with 26 outputs. The transformer acrhitecture is suitabe for the task as it can naturally learn to focus on some local patterns within words via the attention mechanism. In fact, it is common to train LLMs using "masked language modeling", which is similar to the given problem. The proposed procedure is two-stage:
- ***Stage 1:*** we first pre-train a transformer and teach it to identify patterns in words. To this end, akin to masked language modeling, we mask *one* random letter in each word with the goal of predicting it.
- ***Stage 2:*** since the model will often be presented with more than one blanks in one word, we need to finetune it on the distribution of inputs as close to the real game scenario as possible. To this end, for each training word, we randomly mask out all instances of some subcollection of its letters and perform a multi-label classification with the goal of predicting all of those masked letters.

Please see additional comments below for further details.

In [5]:
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel
from nltk.corpus import wordnet
import nltk
nltk.download('wordnet')
import torch.nn.functional as F
import torch
import argparse
import re
import numpy as np

[nltk_data] Downloading package wordnet to /home/amv458/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')
print(f'[device {device} is ready]')

[device cuda:0 is ready]


In [32]:
def build_dictionaries(train_file="words_250000_train.txt"):
    text_file = open(train_file, "r")
    train_dictionary = text_file.read().splitlines()
    text_file.close()
    train_set = set(train_dictionary)
    test_dictionary = []
    for synset in wordnet.all_synsets():
        for lemma in synset.lemmas():
            word = lemma.name().lower()
            if word not in train_set and word.isalpha() :
                test_dictionary.append(word)
    return train_dictionary, test_dictionary


def mean_pooling(token_embeddings, attention_mask):
    """ Average pooling layer for the Transformer.
    """
    token_embeddings = token_embeddings.to(attention_mask.device)
    input_mask_expanded=(attention_mask.unsqueeze(-1).float())
    sum_embeddings=torch.sum(token_embeddings * input_mask_expanded, -2)
    sum_mask=torch.clamp(input_mask_expanded.sum(-2), min=1e-9)
    return sum_embeddings / sum_mask

class HangmanTransformer(torch.nn.Module):
    def __init__(self,
            max_seq_len,
            model_dim,
            latent_dim,
            tokenizer,
            device=torch.device('cpu'),
            num_heads=1,
            num_layers=1,
            vocab_size=28):
        super().__init__()
        bert = BertModel.from_pretrained('bert-base-uncased')
        bert.config.hidden_size=model_dim
        bert.config.vocab_size=vocab_size
        bert.config.num_attention_heads=num_heads
        bert.config.num_hidden_layers=num_layers
        bert.config.pad_token_id=27
        bert.config.intermediate_size=latent_dim
        bert.config.max_position_embeddings=max_seq_len
        self.bert_model = BertModel(bert.config)
        self.classifier = torch.nn.Linear(model_dim, 26)
        self.config = bert.config
        self.guessed_letters = []
        self.alphabet = np.array([chr(97+i) for i in range(26)])
        self.tokenizer = tokenizer
        self.to(device)
        self.device = device
        
    def flush(self):
        """ Resets the Hangman game.
        """
        self.guessed_letters = []
        
    def guess(self, pattern):
        if len(pattern)>self.tokenizer.max_seq_len:
            """ Less than 0.1% of all training words have
                length larger than 20, so we "ignore" those.
            """
            letters_argsort = range(26)
        else:
            inp = self.tokenizer(pattern)
            inp['input_ids'] = inp['input_ids'].unsqueeze(0)
            inp['attention_mask'] = inp['attention_mask'].unsqueeze(0)
            out = F.softmax(self(inp), dim=-1).squeeze()
            letters_argsort = sorted(range(26), key=lambda i: out[i], reverse=True)
        letters = self.alphabet[letters_argsort]
        for letter in letters:
            if letter not in self.guessed_letters:
                self.guessed_letters.append(letter)
                return letter
    
    def forward(self, inp):
        attention_mask = inp['attention_mask'][:, None, None, :].to(self.device)
        input_ids = inp['input_ids'].to(self.device)
        x = self.bert_model.embeddings(input_ids)
        x = self.bert_model.encoder(x, attention_mask=attention_mask)
        x = mean_pooling(x['last_hidden_state'], inp['attention_mask'])
        x = self.classifier(x)
        return x

In [16]:
class HangmanTokenizer():
    def __init__(self, max_seq_len, device):
        self.max_seq_len = max_seq_len
        self.special_idxs = {
            'pad_idx': 27,
            'dot_idx': 26}
        self.device=device
        
    def __call__(self, pattern, word=None):
        """ Tokenizes a given pattern (word).
        """
        inp = torch.zeros(self.max_seq_len)
        attn_mask = torch.zeros(self.max_seq_len)
        fill_mask = torch.zeros(26)
        label = torch.zeros(26)
        for j in range(len(pattern)):
            if pattern[j].isalpha():
                inp[j] = ord(pattern[j])-97
                fill_mask[ord(pattern[j])-97] = -1e9
            else:
                inp[j] = self.special_idxs['dot_idx']
                if word is not None:
                    label[ord(word[j])-97] = 1
                    y = torch.LongTensor([ord(word[j])-97])
                else:
                    y = torch.LongTensor([0])
            attn_mask[j] = 1
        for j in range(len(pattern), self.max_seq_len):
            inp[j] = self.special_idxs['pad_idx']
        out = {'input_ids': inp.long().to(self.device),
               'attention_mask': attn_mask.long().to(self.device),
               'fill_mask': fill_mask.float().to(self.device),
               'label': label.float().to(self.device),
               'y': y.to(self.device)}
        return out
        

class HangmanDatasetStage1(Dataset):
    """ This dataset realizes masking of a
        single letter in a word (for Stage 1).
    """
    def __init__(self, dictionary, tokenizer):
        self.dictionary = []
        for word in dictionary:
            if len(word) <= tokenizer.max_seq_len:
                self.dictionary.append(word)
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.dictionary)
        
    def __getitem__(self, i):
        word = self.dictionary[i]
        idx = np.random.choice(range(len(word)))
        pattern = ""
        for i,letter in enumerate(word):
            if i!=idx:
                pattern += letter
            else:
                pattern += '.'
        return self.tokenizer(pattern, word=word)


class HangmanDatasetStage2(Dataset):
    """ This dataset realizes masking of all instances of
        letters in a random subcollection of
        letters in a word (for Stage 2).
    """
    def __init__(self, dictionary, tokenizer):
        self.dictionary = []
        for word in dictionary:
            if len(word) <= tokenizer.max_seq_len:
                self.dictionary.append(word)
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.dictionary)
        
    def __getitem__(self, i):
        word = self.dictionary[i]
        word_letters = set(list(word))
        if len(word_letters)>1:
            num_open = np.random.choice(range(1,len(word_letters)))
            open_letters = np.random.choice(
                list(word_letters),
                size=num_open,
                replace=False)
            open_letters = set(open_letters.tolist())
        else:
            open_letters = set([])
        pattern = ""
        for letter in word:
            if letter in open_letters:
                pattern += letter
            else:
                pattern += '.'
        return self.tokenizer(pattern, word=word)
        

def collate_fn(batch):
    return {'input_ids': torch.vstack([inp['input_ids'] for inp in batch]),
        'attention_mask': torch.vstack([inp['attention_mask'] for inp in batch]),
        'fill_mask': torch.vstack([inp['fill_mask'] for inp in batch]),
        'label': torch.vstack([inp['label'] for inp in batch]),
        'y': torch.cat([inp['y'] for inp in batch])}

### Stage 1: Pre-training
In this stage, we pre-train ```HangmanTransformer``` using "masked word modeling". We use a learning rate linear warmup for 10 epochs followed by exponential decay. Gamma is computed such that the final learning rate is 50 times smaller than the maximum one (reached at the end of warmup).

In [9]:
train_dictionary, test_dictionary = build_dictionaries()
tokenizer = HangmanTokenizer(max_seq_len=20, device=device)
dataset = HangmanDatasetStage1(train_dictionary, tokenizer)
dataloader = DataLoader(
    dataset,
    batch_size=512,
    shuffle=True,
    collate_fn=collate_fn)
model = HangmanTransformer(
    max_seq_len=20,
    model_dim=512,
    latent_dim=1024,
    tokenizer=tokenizer,
    device=device,
    num_heads=8,
    num_layers=12,
    vocab_size=26+len(tokenizer.special_idxs))
gamma = (1./50)**(1./100)
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=2e-4,
    weight_decay=1e-5)
scheduler_warmup = torch.optim.lr_scheduler.LinearLR(
    optimizer,
    start_factor=0.1,
    end_factor=1.0,
    total_iters=10)
scheduler_decay = torch.optim.lr_scheduler.ExponentialLR(
    optimizer,
    gamma=gamma)
schedulers = [scheduler_warmup, scheduler_decay]
scheduler = torch.optim.lr_scheduler.SequentialLR(
    optimizer,
    schedulers,
    milestones=[10])
criterion = torch.nn.CrossEntropyLoss(reduction='mean')

for epoch in range(100):
    losses = []
    acc = 0.
    for batch in dataloader:
        optimizer.zero_grad()
        y = batch['y']
        out = model(batch)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        acc += sum(torch.argmax(out, dim=-1) == y)
    acc = acc/len(dataset)
    scheduler.step()
    print(f'[epoch: {epoch}][loss/acc: {np.mean(losses):.4f}/{acc:.4f}]')
    
state_dict = model.state_dict()
torch.save(state_dict, 'model-checkpoint-stage-1.pt')

[epoch: 0][loss/acc: 2.7580/0.1544]
[epoch: 1][loss/acc: 2.6367/0.2177]
[epoch: 2][loss/acc: 2.4676/0.3169]
[epoch: 3][loss/acc: 2.3372/0.3681]
[epoch: 4][loss/acc: 2.2381/0.3992]
[epoch: 5][loss/acc: 2.1600/0.4210]
[epoch: 6][loss/acc: 2.0960/0.4416]
[epoch: 7][loss/acc: 2.0418/0.4561]
[epoch: 8][loss/acc: 1.9953/0.4698]


/ext3/miniconda3/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


[epoch: 9][loss/acc: 1.9547/0.4803]
[epoch: 10][loss/acc: 1.9186/0.4916]
[epoch: 11][loss/acc: 1.8847/0.5054]
[epoch: 12][loss/acc: 1.8532/0.5189]
[epoch: 13][loss/acc: 1.8237/0.5290]
[epoch: 14][loss/acc: 1.7957/0.5404]
[epoch: 15][loss/acc: 1.7696/0.5494]
[epoch: 16][loss/acc: 1.7446/0.5607]
[epoch: 17][loss/acc: 1.7210/0.5677]
[epoch: 18][loss/acc: 1.6988/0.5756]
[epoch: 19][loss/acc: 1.6775/0.5835]
[epoch: 20][loss/acc: 1.6573/0.5908]
[epoch: 21][loss/acc: 1.6379/0.5961]
[epoch: 22][loss/acc: 1.6193/0.6039]
[epoch: 23][loss/acc: 1.6013/0.6104]
[epoch: 24][loss/acc: 1.5841/0.6165]
[epoch: 25][loss/acc: 1.5675/0.6218]
[epoch: 26][loss/acc: 1.5517/0.6264]
[epoch: 27][loss/acc: 1.5364/0.6319]
[epoch: 28][loss/acc: 1.5216/0.6360]
[epoch: 29][loss/acc: 1.5075/0.6401]
[epoch: 30][loss/acc: 1.4938/0.6460]
[epoch: 31][loss/acc: 1.4805/0.6489]
[epoch: 32][loss/acc: 1.4677/0.6530]
[epoch: 33][loss/acc: 1.4552/0.6568]
[epoch: 34][loss/acc: 1.4431/0.6615]
[epoch: 35][loss/acc: 1.4314/0.6643]
[e

### Stage 2: Finetuning
Now that we pre-trained a model to recognize word structures and grammar, we finetune it on a more realistic data distribution (with more than one blanks in words) as generated by ```HangmanDatasetStage2```. Now, the objective function is to guess all of the letters that are masked out in the input, so we use a multi-label BCE loss. In addition, we utilize the fact that none of the revealed letters can be masked and effectively "mask" them out in the output by using ```fill_mask``` returned by the tokenizer. To pick the best model during training, we perform validation using a hold-out dictionary.

In [52]:
def validate(model, dictionary, verbose=False, n=100):
    successes = 0.
    words = np.random.choice(dictionary, size=n, replace=False)
    for word in tqdm(words):
        model.flush()
        pattern = ['.']*len(word)
        if verbose:
            print(f'[GAME]: {"".join(pattern)}')
        missed = 0
        while True:
            letter = model.guess("".join(pattern))
            guess_correct = False
            for i in range(len(word)):
                if word[i] == letter:
                    pattern[i] = letter
                    guess_correct = True
            if not guess_correct:
                missed += 1
            if missed == 6:
                if verbose:
                    print(f'[FAIL][word: {word}]')
                break
            elif '.' not in pattern:
                successes += 1
                if verbose:
                    print(f'[OK!][word: {word}]')
                break
            else:
                if verbose:
                    print(f'-[{6-missed}]: {"".join(pattern)} ({letter})')
    return successes/n

In [27]:
state_dict = torch.load('model-checkpoint-stage-1.pt')
model.load_state_dict(state_dict)
gamma = (1./50)**(1./50)
optimizer = torch.optim.Adam(
    [{'params': model.bert_model.parameters(), 'lr': 1e-5},
    {'params': model.classifier.parameters(), 'lr': 1e-4}],
    lr=1e-4)
scheduler_warmup = torch.optim.lr_scheduler.LinearLR(
    optimizer,
    start_factor=0.1,
    end_factor=1.0,
    total_iters=10)
scheduler_decay = torch.optim.lr_scheduler.ExponentialLR(
    optimizer,
    gamma=gamma)
schedulers = [scheduler_warmup, scheduler_decay]
criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')
scheduler = torch.optim.lr_scheduler.SequentialLR(
    optimizer,
    schedulers,
    milestones=[10])
dataset = HangmanDatasetStage2(train_dictionary, tokenizer)
dataloader = DataLoader(
    dataset,
    batch_size=128,
    shuffle=True,
    collate_fn=collate_fn)

best_model = {
    'accuracy': acc,
    'model': None}
for epoch in range(50):
    losses = []
    if epoch % 5 == 0:
        model.eval()
        acc = validate(model, test_dictionary, n=100)
        if best_model['accuracy'] < acc:
            best_model['model'] = model.state_dict()
        torch.save(best_model['model'], 'model-checkpoint-stage-2.pt')
        model.train()
    for batch in dataloader:
        optimizer.zero_grad()
        out = model(batch)
        out = out + batch['fill_mask']
        loss = criterion(out, batch['label'])
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    scheduler.step()
    print(f'[epoch: {epoch}][loss/acc: {np.mean(losses):.4f}/{acc:.4f}]')
    
model.eval()
acc = validate(api, n=100)
if best_model['accuracy'] < acc:
    best_model['model'] = model.state_dict()
torch.save(best_model['model'], 'model-checkpoint-stage-2.pt')

100%|██████████| 100/100 [07:32<00:00,  4.53s/it]


[epoch: 0][loss/acc: 0.3370/0.3300]
[epoch: 1][loss/acc: 0.2825/0.3300]
[epoch: 2][loss/acc: 0.2728/0.3300]
[epoch: 3][loss/acc: 0.2681/0.3300]
[epoch: 4][loss/acc: 0.2648/0.3300]


100%|██████████| 100/100 [07:44<00:00,  4.65s/it]


[epoch: 5][loss/acc: 0.2623/0.5600]
[epoch: 6][loss/acc: 0.2601/0.5600]
[epoch: 7][loss/acc: 0.2586/0.5600]
[epoch: 8][loss/acc: 0.2569/0.5600]
[epoch: 9][loss/acc: 0.2553/0.5600]


100%|██████████| 100/100 [07:38<00:00,  4.59s/it]


...

[epoch: 35][loss/acc: 0.2460/0.6600]
[epoch: 36][loss/acc: 0.2455/0.6600]
[epoch: 37][loss/acc: 0.2453/0.6600]
[epoch: 38][loss/acc: 0.2452/0.6600]
[epoch: 39][loss/acc: 0.2454/0.6600]


100%|██████████| 100/100 [07:44<00:00,  4.64s/it]


[epoch: 40][loss/acc: 0.2452/0.7100]
[epoch: 41][loss/acc: 0.2450/0.7100]
[epoch: 42][loss/acc: 0.2455/0.7100]
[epoch: 43][loss/acc: 0.2454/0.7100]
[epoch: 44][loss/acc: 0.2453/0.7100]


 66%|██████▌   | 66/100 [05:04<02:37,  4.62s/it]


IndexError: index 20 is out of bounds for dimension 0 with size 20

## Verbose Evaluation

In [24]:
tokenizer = HangmanTokenizer(max_seq_len=20, device=device)
final_model = HangmanTransformer(
    max_seq_len=20,
    model_dim=512,
    latent_dim=1024,
    tokenizer=tokenizer,
    device=device,
    num_heads=8,
    num_layers=12,
    vocab_size=26+len(tokenizer.special_idxs))
state_dict = torch.load('model-checkpoint-stage-2.pt')
final_model.load_state_dict(state_dict)
final_model.eval()
print(f'[Model ready]')

[Model ready]


In [74]:
# Multiple eval.

N = 500
word = np.random.choice(test_dictionary, size=N, replace=False)
acc = validate(final_model, word, verbose=False, n=N)
print(f'[accuracy: {acc:.3f}]')

100%|██████████| 500/500 [01:01<00:00,  8.19it/s]

[accuracy: 0.592]


In [77]:
# Single eval.

word = np.random.choice(test_dictionary)
acc = validate(final_model, [word], verbose=True, n=1)

100%|██████████| 1/1 [00:00<00:00,  5.53it/s]

[GAME]: .........
-[6]: .e..e...e (e)
-[6]: .e.re...e (r)
-[6]: .e.re..te (t)
-[6]: .e.re.ate (a)
-[5]: .e.re.ate (c)
-[4]: .e.re.ate (p)
-[3]: .e.re.ate (d)
-[3]: .egregate (g)
[OK!][word: segregate]
